## Version 20240519

## 00 Importing Modules & Mounting Drive

In [ ]:
!pip install matplotlib==3.9.0
!pip install ydata-profiling==4.8.3

In [ ]:
import pathlib
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import sweetviz as sv

from ydata_profiling import ProfileReport
from autoviz import AutoViz_Class
from tabulate import tabulate
from time import sleep

%matplotlib inline

## 01 Choosing & Importing Dataset

### 01.01 Importing Main Data

In [ ]:
dataset_path = 'https://raw.githubusercontent.com/azzindani/00_Data_Source/main/Adidas_US_Sales.csv' # replace this

df = pd.read_csv(dataset_path, encoding = 'ISO-8859-1')#, sep = ';')
df.shape

In [ ]:
df.nunique()

In [ ]:
df.head(2)

In [ ]:
df.info()

### 01.02 Importing Geo Data

In [ ]:
geo_path = 'https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json' # replace this

gdf = gpd.read_file(geo_path)
gdf.head(2) #'''

In [ ]:
gdf = gdf.rename(columns = {'name' : 'State'})
gdf = gdf[['State', 'geometry']]
gdf.head(2)

### 01.03 Importing Additional Data (for enrichment)

## 02 Data Structuring

### 02.01 Selecting & Dropping Variables

In [ ]:
column_list = [] # fill this

df = df.drop(column_list, axis = 1)
df.head(2) #'''

### 02.02 Cleaning Text Object

#### 02.02.01 Convert Header to Proper Text

In [ ]:
for x in df.columns:
  y = x.title()
  df = df.rename(columns = {x : y}) #'''

#### 02.02.02 Strip Abnormal Spaces

In [ ]:
for x in df.columns:
  if df[x].dtypes == 'object':
    try:
      df[x] = df[x].str.strip()
    except:
      pass #'''

#### 02.02.03 Convert Object Content to Proper Text

In [ ]:
for x in df.columns:
  if df[x].dtypes == 'object':
    for a in df[x].unique():
      b = a.title()
      df[x] = df[x].replace(a, b)
  else:
    pass #'''

### 02.03 Coverting Data

#### 02.03.01 Convert to date

In [ ]:
df.head(2)

In [ ]:
column_list = ['Activity Period Start Date'] # fill this

for x in column_list:
  try:
    df[x] = pd.to_datetime(df[x])
  except:
    pass #'''

#### 02.03.02 Convert to integer

In [ ]:
column_list = [] # fill this

for x in column_list:
  df[x] = df[x].astype('int') #'''

#### 02.03.03 Convert to Object (if necessary)

In [ ]:
column_list = [] # fill this

for x in column_list:
  df[x] = df[x].astype('str') #'''

#### 02.03.04 Replace 0 to Nan (if necessary)

In [ ]:
#df = df.replace(0, np.nan)

## 03 Data Cleaning

### 03.01 Replacing Variable Component

In [ ]:
'''value_dict = {} # fill this

column_name = ''

df[column_name] = df[column_name].replace(value_dict) #'''

### 03.02 Add New Variable

#### 03.02.01 Add by Math Calculation

In [ ]:
'''new_var = '' # fill this
obj_var1 = '' # fill this
obj_var2 = '' # fill this

df[new_var] = df[obj_var1] * df [obj_var2] #'''

#### 03.02.02 Add by Replacing "Other"

In [ ]:
'''column_name = '' # fill this

df[column_name].value_counts() #'''

In [ ]:
'''value_thres = 1300

replace_list = []

df_dict = df[column_name].value_counts().to_dict()
for i in df_dict:
  if df_dict[i] < value_thres:
    replace_list.append(i)

replace_list #'''

In [ ]:
'''df[column_name] = df[column_name].copy().replace(to_replace = replace_list, value = 'Other') #'''

### 03.03 Inaccuracies

In [ ]:
column_list = [] # fill this

for x in column_list:
  try:
    df[x] = df[x].replace('0', np.nan)
    df = df.dropna()
  except:
    pass

### 03.04 Handling Outliers
##### Data Distribution Check (Before)

In [ ]:
df.head(2)

In [ ]:
df.info()

In [ ]:
'''x = 'Geo Region' # replace this
y = 'Cargo Weight Lbs' # replace this
color = 'Operating Airline' # replace this

fig = px.histogram(
  df,
  x = x,
  y = y,
  #color = color,
  marginal = 'box',
  hover_data = df.columns
)

fig.show() #'''

In [ ]:
'''x = 'Activity Period' # replace this
y = 'Cargo Weight Lbs' # replace this
color = 'Operating Airline' # replace this

fig = px.scatter(
  df,
  x = x,
  y = y,
  #color = color,
  hover_data = df.columns
)

fig.show() #'''

#### 03.04.01 Using IQR (Inter Quantile Range)

In [ ]:
def iqr_thres(dataframe, column, th1 = 0.25, th3 = 0.75):
  quartile1 = dataframe[column].quantile(th1)
  quartile3 = dataframe[column].quantile(th3)
  iqr = quartile3 - quartile1
  upper_limit = quartile3 + 1.5 * iqr
  lower_limit = quartile1 - 1.5 * iqr

  return lower_limit, upper_limit

In [ ]:
def check_outliers_iqr(dataframe, column):
  lower_limit, upper_limit = iqr_thres(dataframe, column)
  if dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)].any(axis = None):
    return True
  else:
    return False

In [ ]:
def replace_iqr(dataframe, columns, th1 = 0.25, th3 = 0.75, replace = False):
  data = []

  for column in columns:
    if df[column].dtypes == 'int64' or df[column].dtypes == 'float64':
      if column != 'Outcome':
        outliers_ = check_outliers_iqr(df, column)
        count = None
        lower_limit, upper_limit = iqr_thres(dataframe, column, th1, th3)

        if outliers_:
          count = dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)][column].count()
          if replace:
            if lower_limit < 0:
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
            else:
              dataframe.loc[(dataframe[column] < lower_limit), column] = lower_limit
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
        outliers_status = check_outliers_iqr(df, column)
        data.append([outliers_, outliers_status, count, column, lower_limit, upper_limit ])

  table = tabulate(data, headers = ['Outliers (Previously)', 'Outliers', 'Count', 'Column', 'Lower Limit', 'Upper Limit'], tablefmt = 'rst', numalign = 'right')
  print('Removing Outliers using IQR')
  print(table)

In [ ]:
column_list = [] # fill this

replace_iqr(
  dataframe = df,
  columns = column_list,
  replace = True
) #'''

#### 03.04.02 Using Standard Deviation

In [ ]:
def std_thres(dataframe, column):
  upper_limit = dataframe[column].mean() + 3 * dataframe[column].std()
  lower_limit = dataframe[column].mean() - 3 * dataframe[column].std()

  return lower_limit, upper_limit

In [ ]:
def check_outliers_std(dataframe, column):
  lower_limit, upper_limit = iqr_thres(dataframe, column)
  if dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)].any(axis = None):
    return True
  else:
    return False

In [ ]:
def replace_std(dataframe, columns, replace = False):
  data = []

  for column in columns:
    if df[column].dtypes == 'int64' or df[column].dtypes == 'float64':
      if column != 'Outcome':
        outliers_ = check_outliers_std(dataframe, column)
        count = None
        lower_limit, upper_limit = std_thres(dataframe, column)

        if outliers_:
          count = dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)][column].count()
          if replace:
            if lower_limit < 0:
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
            else:
              dataframe.loc[(dataframe[column] < lower_limit), column] = lower_limit
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
        outliers_status = check_outliers_std(dataframe, column)
        data.append([outliers_, outliers_status, count, column, lower_limit, upper_limit])

  table = tabulate(data, headers = ['Outlier (Previously)', 'Outliers', 'Count', 'Column', 'Lower Limit', 'Upper Limit'], tablefmt = 'rst', numalign = 'right')
  print('Removing Outliers using 3 Standard Deviation')
  print(table)

In [ ]:
column_list = [] # fill this

replace_iqr(
  dataframe = df,
  columns = column_list,
  replace = False
) #'''

### 03.05 Handling Missing / Zeros / Null
##### Filling missing value (numerical only) is better using median than mean or mode

#### 03.05.01 Detecting Zero Value

In [ ]:
zero_columns = [] # fill this

for x in df.columns:
  if df[x].dtypes == 'int64' or df[x].dtypes == 'float64':
    if (df[x] == 0).sum() != 0:
      print(x, ':', str((df[x] == 0).sum()))
      zero_columns.append(x)

zero_columns #'''

#### 03.05.02 Detecting Nan / Non Available Value

In [ ]:
nan_columns = [] # fill this

for x in df.columns:
  if df[x].dtypes == 'object':
    if (df[x] == np.nan).sum() or (df[x] == '-').sum() != 0:
      print(x, ':', str((df[x] == np.nan).sum()))
      nan_columns.append(x)

nan_columns #'''

#### 03.05.03 Replacing Zero with Mean (for numerical value if median value == 0), if necessary

In [ ]:
column_list = [] # fill this

df = df.replace(0, np.nan)

for x in column_list:
  df[x] = df[x].fillna(df[x].mean()) #'''

#### 03.05.04 Replacing Zero with Median (for numerical value if median value != 0), if necessary

In [ ]:
column_list = [] # fill this

df = df.replace(0, np.nan)

for x in column_list:
  df[x] = df[x].fillna(df[x].median()) #'''

#### 03.05.05 Replacing Zero with Mode (for categorical / object value), if necessary

In [ ]:
column_list = [] # fill this

df = df.replace(0, np.nan)

for x in column_list:
  df[x] = df[x].fillna(df[x].mode()) #'''

### 03.06 Handling Incomplete Data

### 03.07 Handling Data Biases

### 03.08 Handling Duplicates

In [ ]:
#df = df.drop_duplicates()

##### Data Distribution Check (After)

In [ ]:
'''x = 'Geo Region' # replace this
y = 'Cargo Weight Lbs' # replace this
color = 'Operating Airline' # replace this

fig = px.histogram(
  df,
  x = x,
  y = y,
  #color = color,
  marginal = 'box',
  hover_data = df.columns
)

fig.show() #'''

In [ ]:
'''x = 'Activity Period' # replace this
y = 'Cargo Weight Lbs' # replace this
color = 'Operating Airline' # replace this

fig = px.scatter(
  df,
  x = x,
  y = y,
  #color = color,
  hover_data = df.columns
)

fig.show() #'''

## 04 Enriching Data
#### take other dataset, inside or outside from related dataset / business

In [ ]:
'''location = 'Country' # replace this

a = df[location].unique()
b = gdf[location].unique()

for i in a:
  if i not in b:
    print(i)

print(50 * '=')

for i in b:
  if i not in a:
    print(i) #'''

In [ ]:
value_dict = {}

try:
  df[location] = df[location].replace(value_dict)
except:
  pass #'''

## 05 Data Validation
#### Verifying consistency, quality, and security of data

## 06 Exploration Data Analysis (Univariate)

### 06.01 Generate Data Profile (YData Profiling)

#### 06.01.01 Report Generation

In [ ]:
description = 'This profiling report was generated for Analytics' # replace this
copyright_holder = 'Mine' # replace this
copyright_year = '2024' # replace this
url = 'https://www.google.com' # replace this
title = 'Sales'

data_profile = ProfileReport(
  df,
  title = title,
  dataset = {
    'description' : description,
    'copyright_holder' : copyright_holder,
    'copyright_year' : copyright_year,
    'url' :  url,
  },
  correlations = {
    'pearson' : {'calculate' : True},
    'spearman' : {'calculate' : True},
    'kendall' : {'calculate' : True},
    'phi_k' : {'calculate' : True},
    'cramers': {'calculate' : True},
  },
)

data_profile #'''

#### 06.01.02 Save Report

In [ ]:
#export = data_profile.to_file('YDATA_PROFILING_REPORT.html')
#export = data_profile.to_file(title + '.json')

### 06.02 SweetViz

#### 06.02.01 Report Generation

In [ ]:
sweetviz = sv.analyze(df)
sweetviz.show_notebook()

#### 06.02.02 Save Report

In [ ]:
#sweetviz.show_html()

### 06.03 AutoViz

#### 06.03.01 Report Generation

In [ ]:
autoviz = AutoViz_Class().AutoViz(df)

In [ ]:
try:
  autoviz = AutoViz_Class().AutoViz(df, chart_format = 'svg')
except:
  pass

In [ ]:
try:
  autoviz = AutoViz_Class().AutoViz(df, chart_format = 'bokeh')
except:
  pass

#### 06.03.02 Save Report

In [ ]:
'''try:
  autoviz = AutoViz_Class().AutoViz(df, chart_format = 'html')
except:
  pass #'''

### 06.04 Lux
step by step manual read : https://lux-api.readthedocs.io/en/latest/source/guide/export.html

In [ ]:
'''import lux

df.head(1000000) #'''

## 07 Cache Data (if needed)

In [ ]:
'''filename = '_cache' # replace this

cache_dir = CACHE_PATH + '/' + filename + '.csv'
print(cache_dir)
df.to_csv(cache_dir, index = False) #'''

## 08 Table, Chart Drafting & Testing

### 08.01 Template Setup

In [ ]:
colors_1 = px.colors.sequential.Reds
colors_2 = px.colors.sequential.RdBu
explode = tuple([0.015] * 50)
latitude = 37.09024
longitude = -73.935242
chart_theme = 'plotly_dark'
streamlit_theme = 'streamlit'
margin = {'r' : 20, 't' : 40, 'l' : 20, 'b' : 10}
cmap = 'magma_r'
title_x = 0
title_font_size = 18

In [ ]:
filtered_df = df

### 08.02 Create Main Chart

In [ ]:
instruction = 'sum' # or count or mean

for x in filtered_df.columns:
  if filtered_df[x].dtypes == 'int64' or filtered_df[x].dtypes == 'float64':
    try:
      if instruction == 'sum':
        t = 'Total of '
        value = filtered_df[x].sum()

      if instruction == 'count':
        t = 'Frequency of '
        value = filtered_df[x].count()

      if instruction == 'mean':
        t = 'Average of '
        value = filtered_df[x].mean()
      title = t + x

      fig = go.Figure(go.Indicator(
        mode = 'number+delta',
        domain = {'x': [0, 1], 'y': [0, 1]},
        value = value,
      ))
      fig.update_layout(
        height = 200,
        margin = margin,
        title = title,
        titlefont = dict(size = title_font_size),
        title_x = title_x,
      )
      fig.show()

    except:
      pass #'''

### 08.03 Create Bar Chart

In [ ]:
num_var = []
cat_var = []

for x in filtered_df.columns:
  if filtered_df[x].dtypes == 'int64' or filtered_df[x].dtypes == 'float64':
    num_var.append(x)

for x in filtered_df.columns:
  if filtered_df[x].dtypes == 'object':
    cat_var.append(x)

In [ ]:
n = None
instruction = 'count' # or count or mean

for x in num_var[:n]:
  for y in cat_var[:n]:
    try:
      if instruction == 'sum':
        t = 'Total of '

        chart_df_3 = filtered_df.groupby(by = [y], as_index = False,)[[x]].sum()
        chart_df_3 = chart_df_3.sort_values(by = x, ascending = False)

      if instruction == 'count':
        t = 'Frequency of '

        chart_df_3 = filtered_df.groupby(by = [y], as_index = False,)[[x]].count()
        chart_df_3 = chart_df_3.sort_values(by = x, ascending = False)

      if instruction == 'mean':
        t = 'Average of '

        chart_df_3 = filtered_df.groupby(by = [y], as_index = False,)[[x]].mean() # or count() or mean()
        chart_df_3 = chart_df_3.sort_values(by = x, ascending = False)

      title = t + x + ' by ' + y

      fig = px.bar(
        chart_df_3,
        x = x, # can be switched x or y variable
        y = y, # can be switched x or y variable
        template = chart_theme,
        color_discrete_sequence = colors_2,
        title = title,
        text_auto = ',.0f',
      )
      fig.update_layout(
          height = 400,
          margin = margin,
          titlefont = dict(size = title_font_size),
          title_x = title_x,
      )
      fig.update_yaxes(categoryorder = 'total ascending')
      fig.show()

    except:
      pass #'''

### 08.04 Create Pie Chart

In [ ]:
n = 2
instruction = 'count' # or count or mean

for x in num_var[:n]:
  for y in cat_var[:n]:
    try:
      if instruction == 'sum':
        t = 'Total of '

        chart_df_3 = filtered_df.groupby(by = [y], as_index = False,)[[x]].sum()

      if instruction == 'count':
        t = 'Frequency of '

        chart_df_3 = filtered_df.groupby(by = [y], as_index = False,)[[x]].count()

      if instruction == 'mean':
        t = 'Average of '

        chart_df_3 = filtered_df.groupby(by = [y], as_index = False,)[[x]].mean()

      title = t + x + ' by ' + y + ' Ratio'

      fig = px.pie(
        chart_df_3,
        values = x,
        names = y,
        hole = 0.5,
        template = chart_theme,
        color_discrete_sequence = colors_2,
      )
      fig.update_traces(
        text = chart_df_3[y],
        textposition = 'inside',
        #pull = explode,
      )
      fig.update_layout(
        height = 400,
        margin = margin,
        title = title,
        titlefont = dict(size = title_font_size),
        title_x = title_x,
      )
      fig.show()

    except:
      pass #'''

### 08.05 Create Geospatial Chart

In [ ]:
n = 1
instruction = 'sum' # or count or mean
var_label = 'State' # replace this

for var_number in num_var[:n]:
  try:
    if instruction == 'sum':
      t = 'Total of '

      map_df = filtered_df.groupby([var_label])[var_number].sum()

    if instruction == 'count':
      t = 'Frequency of '

      map_df = filtered_df.groupby([var_label])[var_number].count()

    if instruction == 'mean':
      t = 'Average of '

      map_df = filtered_df.groupby([var_label])[var_number].mean()

    title = t + var_number + ' by ' + var_label

    map_df = filtered_df.groupby([var_label])[var_number].sum()
    map_df = map_df.reset_index()
    map_df = gdf.merge(map_df, on = var_label)

    map_df = map_df.set_index(var_label)
    map_df.head(3)

    fig = px.choropleth_mapbox(
      data_frame = map_df,
      geojson = map_df.geometry,
      locations = map_df.index,
      color = var_number,
      color_continuous_scale = colors_1,
      #range_color = (0, 10),
      opacity = 0.5,
      center = {'lat' : latitude, 'lon' : longitude},
      mapbox_style = 'carto-positron',
      zoom = 4,
    )
    fig.update_geos(
      fitbounds = 'locations',
      visible = False,
    )
    fig.update_layout(
      height = 600,
      margin = margin,
      title = title,
      titlefont = dict(size = title_font_size),
      title_x = title_x,
    )
    fig.show()

  except:
    pass #'''

### 08.06 Create Time Series Chart

In [ ]:
n = 2
p = 'Activity Period Start Date' # period
x = 'Month & Year' # x axis variable

for y in num_var[:n]:
  for c in cat_var[:n]:
    try:
      filtered_df[x] = filtered_df[p].dt.to_period('M') # Y = Year, Q = Quarter, M = Month, W = Week, D = Day
      linechart = pd.DataFrame(filtered_df.groupby([x, c])[[y]].count()).reset_index()
      linechart[x] = linechart[x].astype(str)

      title = 'Monthly ' + y + ' by ' + c
      fig = px.line(
        x = linechart[x],
        y = linechart[y],
        color = linechart[c],
        markers = True,
      )
      fig.update_traces(
        #hovertemplate = '{:,.0f}'
      )
      fig.update_layout(
        hovermode = 'x',
        height = 400,
        margin = margin,
        title = title,
        titlefont = dict(size = title_font_size),
        title_x = title_x,
        yaxis = dict(tickformat = ',.0f'),
      )
      fig.show()

    except:
      pass #'''

### 08.07 Create Treemap Chart

In [ ]:
df.head(2)

In [ ]:
var_num = 'Total Sales' # replace this
categories = ['Region', 'State', 'City'] # replace this
title = 'Hierarchial View of ' + var_num + ' using Tree Map' # replace this

try:
  fig = px.treemap(
    filtered_df,
    path = categories,
    values = var_num,
    hover_data = [var_num],
    color = var_num,
    template = chart_theme,
    color_continuous_scale = colors_2,
  )
  fig.update_layout(
    height = 400,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  fig.show()

except:
  pass #'''

### 08.08 Create Scatter Plot Chart

In [ ]:
'''n = 2

for var_a in num_var[:n]:
  for var_b in num_var[:n]:
    try:
      title = 'Relationship between ' + var_a + ' and ' + var_b + ' using Scatter Plot'

      fig = px.scatter(
        filtered_df,
        x = var_a,
        y = var_b,
        #color = category,
        height = 600,
        template = chart_theme,
        color_discrete_sequence = colors_2,
        trendline = 'ols',
        marginal_x = 'histogram',
        marginal_y = 'box',
      )
      fig.update_layout(
        title = title,
        titlefont = dict(size = title_font_size),
        xaxis = dict(title = var_a, titlefont = dict(size = 14), tickformat = '.2s'),
        yaxis = dict(title = var_b, titlefont = dict(size = 14)),
        title_x = title_x,
      )
      fig.show()

    except:
      pass #'''

In [ ]:
n = 2

for var_a in num_var[:n]:
  for var_b in num_var[:n]:
    for category in cat_var[:n]:
      try:
        title = 'Relationship between ' + var_a + ' and ' + var_b + ' using Scatter Plot'

        fig = px.scatter(
          filtered_df,
          x = var_a,
          y = var_b,
          color = category,
          height = 600,
          template = chart_theme,
          color_discrete_sequence = colors_2,
          trendline = 'ols',
          marginal_x = 'histogram',
          marginal_y = 'box',
        )
        fig.update_layout(
          title = title,
          titlefont = dict(size = title_font_size),
          xaxis = dict(title = var_a, titlefont = dict(size = 14), tickformat = '.2s'),
          yaxis = dict(title = var_b, titlefont = dict(size = 14)),
          title_x = title_x,
        )
        fig.show()

      except:
        pass #'''

In [ ]:
'''var_a = 'price'
var_b = 'Year Built'
category = 'category'

try:
  title = 'Relationship between ' + var_a + ' and ' + var_b + ' using Scatter Plot'

  fig = px.scatter(
    filtered_df,
    x = var_a,
    y = var_b,
    color = category,
    height = 600,
    template = chart_theme,
    color_discrete_sequence = colors_2,
    trendline = 'ols',
    marginal_x = 'histogram',
    marginal_y = 'box',
  )
  fig.update_layout(
    title = title,
    titlefont = dict(size = title_font_size),
    xaxis = dict(title = var_a, titlefont = dict(size = 14), tickformat = '.2s'),
    yaxis = dict(title = var_b, titlefont = dict(size = 14)),
    title_x = title_x,
  )
  fig.show()

except:
  pass #'''

### 08.09 Create Radius Chart

In [ ]:
'''cat_var = 'Satisfaction' # replace this

score_vars = [] # replace this

try:
  chart_df_1 = filtered_df.groupby(by = cat_var, as_index = False,)[score_vars].mean()

  title = cat_var + ' Diagram'

  fig = go.Figure()
  fig.add_trace(go.Scatterpolar(
    r = chart_df_1.loc[1].to_list()[1:],
    theta = chart_df_1.columns[1:],
    fill = 'toself',
    name = cat_var,
    fillcolor = colors_1[1],
    opacity = 0.6,
    line = dict(color = colors_1[2]),
  ))
  fig.update_layout(
    polar = dict(
      radialaxis = dict(
        visible = True,
        range = [0, 5]
      )),
    showlegend = False
  )
  fig.update_layout(
    height = 500,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  fig.show()

except:
  pass #'''

### 08.10 Create Custom Bar Chart
used for creating chart with >1 variables or columns

In [ ]:
'''var_a = 'Sales'
var_b = 'Profit'
var_label = 'Market'

try:
  title = 'Comparison between ' + var_a + ' and ' + var_b + ' by ' + var_label

  fig = go.Figure(data = [
    go.Bar(
      name = var_a,
      x = filtered_df[var_label],
      y = filtered_df[var_a],
      text = ['${:,.2f}'.format(x) for x in filtered_df[var_a]]
    ),
    go.Bar(
      name = var_b,
      x = filtered_df[var_label],
      y = filtered_df[var_b],
      text = ['${:,.2f}'.format(x) for x in filtered_df[var_b]]
    ),
  ])
  fig.update_layout(
    hovermode = 'x',
    height = 400,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  fig.show()

except:
  pass #'''

### 08.11 Create Custom Line Chart
used for creating chart with >1 variables or columns

In [ ]:
'''var_label = 'Month & Year'
var_a = 'Sales'
var_b = 'Profit'

try:
  linechart = pd.DataFrame(filtered_df.groupby(filtered_df[var_label])[[var_a, var_b]].sum()).reset_index()
  linechart[var_label] = linechart[var_label].astype(str)

  title = 'Time Series ' + var_a + ' and ' + var_b

  fig = go.Figure()
  fig.add_trace(go.Scatter(
    x = linechart[var_label],
    y = linechart[var_a],
    mode = 'lines+markers',
    name = var_a,
    marker = {'color': colors_2[0]},
  ))
  fig.add_trace(go.Scatter(
    x = linechart[var_label],
    y = linechart[var_b],
    mode = 'lines+markers',
    name = var_b,
    marker = {'color': colors_2[2]},
  ))
  fig.update_layout(
    hovermode = 'x',
    height = 400,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  fig.show()

except:
  pass #'''

## 09 Create Dashboard
### * template : ['ggplot2', 'seaborn', 'simple_white', 'plotly', 'plotly_white', 'plotly_dark', 'presentation', 'xgridoff', 'ygridoff', 'gridon', 'none']
### * change layout https://plotly.com/python-api-reference/generated/plotly.graph_objects.Layout.html
### * more chart customization : https://plotly.com/python-api-reference/index.html

In [ ]:
%%writefile app.py

import streamlit as st
import os
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import warnings
warnings.filterwarnings('ignore')

# 00 CREATING TAB TITLE

st.set_page_config(
  page_title = 'Bike Sales',
  page_icon = ':bar_chart:',
  layout = 'wide',
)


# 01 CREATING DASHBOARD TITLE

st.title(':bar_chart: Bike Sales Dashboard')
st.markdown('<style>div,block-container{padding-top:0rem;}<style>', unsafe_allow_html = True)


# 02 IMPORTING DATASET

dataset_path = 'https://raw.githubusercontent.com/azzindani/00_Data_Source/main/Europe_Bike_Sales.csv'
df = pd.read_csv(dataset_path, encoding = 'ISO-8859-1')

df['Country'] = df['Country'].replace({
    'United States' : 'United States of America',
})

gdf = gpd.read_file('https://raw.githubusercontent.com/azzindani/00_Data_Source/main/Countries_Geojson.geojson')
gdf = gdf.rename(columns = {'admin' : 'Country'})
gdf = gdf[['Country', 'adm0_a3', 'geometry']]


# 03 SETUP TEMPLATE & THEME

colors_1 = px.colors.sequential.Rainbow
colors_2 = px.colors.sequential.Rainbow
explode = tuple([0.015] * 50)
latitude = 0
longitude = 0
chart_theme = 'plotly_dark'
streamlit_theme = 'streamlit'
margin = {'r' : 20, 't' : 40, 'l' : 20, 'b' : 10}
cmap = 'rainbow'
title_x = 0
title_font_size = 18


# 04 CREATING DATE PICKER

col_1, col_2 = st.columns((2))
df['Date'] = pd.to_datetime(df['Date'])

# Getting min & max date
start_date = pd.to_datetime(df['Date']).min()
end_date = pd.to_datetime(df['Date']).max()

with col_1:
  date_1 = pd.to_datetime(st.date_input('Start Date', start_date))

with col_2:
  date_2 = pd.to_datetime(st.date_input('End Date', end_date))

df = df[(df['Date'] >= date_1) & (df['Date'] <= date_2)] #'''


# 05 CREATING SIDEBAR FILTER

st.sidebar.header('Choose your Filter: ')

# Create country sidebar
country = st.sidebar.multiselect('Select Country', df['Country'].unique())

if not country:
  df_2 = df.copy()
else:
  df_2 = df[df['Country'].isin(country)]

# Create state sidebar
state = st.sidebar.multiselect('Select State', df_2['State'].unique())


# 06 CREATING DATASET FILTER LOGIC

# Filter the data based on country, state

# None filter
if not country and not state:
  filtered_df = df

# Single filter
elif not state:
  filtered_df = df[df['Country'].isin(country)]

elif not county:
  filtered_df = df[df['State'].isin(state)]

else:
  filtered_df = df_2[df_2['Country'].isin(country) & df_2['State'].isin(state)]


# 07 CREATING DASHBOARD

# create highlighted indicator

col_11, col_12, col_13 = st.columns((1, 1, 3))

with col_11:
  title = 'Revenue'
  #st.subheader(title)
  fig = go.Figure(go.Indicator(
    mode = 'number+delta',
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = filtered_df['Revenue'].sum(),
    number = {'prefix' : '$'},
  ))

  fig.update_layout(
    #paper_bgcolor = 'lightgray',
    height = 200,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )

  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

with col_12:
  title = 'Units Sold'
  #st.subheader(title)
  fig = go.Figure(go.Indicator(
    mode = 'number+delta',
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = filtered_df['Order_Quantity'].sum(),
    number = {'valueformat' : ','},
  ))

  fig.update_layout(
    #paper_bgcolor = 'lightgray',
    height = 200,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )

  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

# create geospatial chart

map_df = filtered_df.groupby(['Country'])['Revenue'].sum()
map_df = map_df.reset_index()
map_df = map_df.rename(columns = {'Revenue' : 'Revenue (USD)'})
map_df = gdf.merge(map_df, on = 'Country')

var_label = 'Country'
var_number = 'Revenue (USD)'

map_df = map_df.set_index(var_label)

with col_13:
  title = 'Geospatial Sales Data'
  fig = px.choropleth_mapbox(
      data_frame = map_df,
      geojson = map_df.geometry,
      locations = map_df.index,
      color = var_number,
      color_continuous_scale = colors_1,
      #range_color = (0, 10),
      opacity = 0.5,
      center = {'lat' : latitude, 'lon' : longitude},
      mapbox_style = 'carto-positron',
      zoom = 0.3,
  )
  fig.update_geos(
      fitbounds = 'locations',
      visible = False,
  )
  fig.update_layout(
      height = 600,
      margin = margin,
      title = title,
      titlefont = dict(size = title_font_size),
      title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

# create market outlook chart

# create pie chart

chart_df_1 = filtered_df.groupby(by = ['Age_Group'], as_index = False,)[['Order_Quantity']].sum()

with col_11:
  title = 'Unit Sold by Age Group'
  fig = px.bar(
    chart_df_1,
    y = 'Age_Group',
    x = 'Order_Quantity',
    title = title,
    color_discrete_sequence = colors_1,
    text_auto = ',.0f',
  )
  fig.update_traces(
      textfont_size = 12,
      textangle = 0,
      textposition = 'inside',
      cliponaxis = False
  )

  fig.update_xaxes(showspikes = True)
  fig.update_yaxes(showspikes = True)
  fig.update_layout(
    height = 300,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_1.style.background_gradient(cmap = cmap))
    csv = chart_df_1.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

chart_df_2 = filtered_df.groupby(by = ['Product_Category'], as_index = False)[['Order_Quantity']].sum()

with col_12:
  title = 'Unit Sold by Product Category'
  fig = px.bar(
    chart_df_2,
    y = 'Product_Category',
    x = 'Order_Quantity',
    title = title,
    color_discrete_sequence = colors_1,
    text_auto = ',.0f',
  )
  fig.update_traces(
      textfont_size = 12,
      textangle = 0,
      textposition = 'inside',
      cliponaxis = False
  )

  fig.update_xaxes(showspikes = True)
  fig.update_yaxes(showspikes = True)
  fig.update_layout(
    height = 300,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_2.style.background_gradient(cmap = cmap))
    csv = chart_df_2.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

col_21, col_22, col_23 = st.columns((1, 1, 1))

chart_df_3 = filtered_df.groupby(by = ['Country'], as_index = False,)[['Revenue']].sum()

with col_21:
  title = 'Revenue Ratio by Country'
  #st.subheader(title)
  fig = px.pie(
    chart_df_3,
    values = 'Revenue',
    names = 'Country',
    hole = 0.5,
    template = chart_theme,
    color_discrete_sequence = colors_1,
  )
  fig.update_traces(
    text = chart_df_3['Country'],
    textposition = 'inside',
    pull = explode,
  )
  fig.update_layout(
    height = 400,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_3.style.background_gradient(cmap = cmap))
    csv = chart_df_3.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

chart_df_4 = filtered_df.groupby(by = ['Country', 'Product_Category'], as_index = False)[['Revenue', 'Profit']].sum()

with col_22:
  title = 'Product Category by Country'
  fig = px.bar(
    chart_df_4,
    x = 'Country',
    y = 'Revenue',
    text = ['${:,.0f}'.format(x) for x in chart_df_4['Revenue']],
    color = 'Product_Category',
    template = chart_theme,
    color_discrete_sequence = colors_2,
    barmode = 'group',
    height = 500,
  )
  fig.update_traces(
      textfont_size = 12,
      textangle = 0,
      textposition = 'inside',
      cliponaxis = False
  )

  fig.update_xaxes(showspikes = True)
  fig.update_yaxes(showspikes = True)
  fig.update_layout(
    height = 400,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_4.style.background_gradient(cmap = cmap))
    csv = chart_df_4.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

filtered_df['Profit Ratio'] = filtered_df['Profit'] / filtered_df['Revenue']
chart_df_5 = filtered_df.groupby(by = ['Sub_Category'], as_index = False,)[['Profit Ratio']].mean()

with col_23:
  title = 'Profit Ratio by Sub Category'
  #st.subheader(title)
  fig = px.bar(
    chart_df_5,
    x = 'Sub_Category',
    y = 'Profit Ratio',
    #text = ['{:,.2f}'.format(x) for x in chart_df_5['Profit Ratio']],
    template = chart_theme,
    color_discrete_sequence = colors_2,
    title = title,
    text_auto = '.2%',
  )
  fig.update_layout(
      height = 400,
      margin = margin,
      titlefont = dict(size = title_font_size),
      title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_5.style.background_gradient(cmap = cmap))
    csv = chart_df_5.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

chart_df_6 = filtered_df.groupby(by = ['Sub_Category'], as_index = False,)[['Revenue', 'Profit']].sum()

title = 'Sales & Operating Profit by Product'
#st.subheader(title)
fig = go.Figure(data = [
  go.Bar(
    name = 'Revenue',
    x = chart_df_6['Sub_Category'],
    y = chart_df_6['Revenue'],
    text = ['${:,.0f}'.format(x) for x in chart_df_4['Revenue']],
    marker = {'color': colors_2[0]}),
  go.Bar(
    name = 'Profit',
    x = chart_df_6['Sub_Category'],
    y = chart_df_6['Profit'],
    text = ['${:,.0f}'.format(x) for x in chart_df_6['Profit']],
    marker = {'color': colors_2[2]}),
])

fig.update_layout(
    template = chart_theme,
    #paper_bgcolor = 'LightSteelBlue',
    height = 400,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
)
st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

with st.expander('View Data'):
  st.write(chart_df_6.style.background_gradient(cmap = cmap))
  csv = chart_df_6.to_csv(index = False).encode('utf-8')
  st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

filtered_df['Profit Ratio'] = filtered_df['Profit'] / filtered_df['Revenue']

filtered_df['Month & Year'] = filtered_df['Date'].dt.to_period('M')
linechart = pd.DataFrame(filtered_df.groupby(df['Month & Year'])[['Revenue', 'Profit']].sum()).reset_index()
linechart['Month & Year'] = linechart['Month & Year'].astype(str)

# create time series chart

title = 'Time Series Sales & Profit Data'
#st.subheader(title)
fig_1 = go.Figure()
fig_1.add_trace(go.Scatter(
  x = linechart['Month & Year'],
  y = linechart['Revenue'],
  mode = 'lines+markers',
  name = 'Revenue',
  marker = {'color': colors_2[0]},
))
fig_1.add_trace(go.Scatter(
  x = linechart['Month & Year'],
  y = linechart['Profit'],
  mode = 'lines+markers',
  name = 'Profit',
  marker = {'color': colors_2[2]},
))
fig_1.update_layout(
  hovermode = 'x',
  height = 400,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
)
st.plotly_chart(fig_1, use_container_width = True, theme = streamlit_theme)

# create treemap chart

title = 'Hierarchial View of Sales using Tree Map'
#st.subheader(title)
fig_2 = px.treemap(
  filtered_df,
  path = ['Country', 'Product_Category', 'Sub_Category'],
  values = 'Revenue',
  hover_data = ['Revenue'],
  color = 'Revenue',
  template = chart_theme,
  color_continuous_scale = colors_2,
)
fig_2.update_layout(
  height = 400,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
)

st.plotly_chart(fig_2, use_container_width = True, theme = streamlit_theme)

# create scatter plot chart

title = 'Relationship between Product Unit Price and Unit Sold using Scatter Plot'
fig_3 = px.scatter(
  filtered_df,
  x = 'Unit_Price',
  y = 'Customer_Age',
  color = 'Product_Category',
  height = 600,
  template = chart_theme,
  color_discrete_sequence = colors_1,
  #trendline = 'ols',
  marginal_x = 'histogram',
  #marginal_y = 'box',
  facet_col = 'Customer_Gender'
)
fig_3.update_layout(
  title = title,
  titlefont = dict(size = title_font_size),
  xaxis = dict(title = 'Unit Price (USD)', titlefont = dict(size = 14)),
  yaxis = dict(title = 'Customer_Age', titlefont = dict(size = 14)),
  title_x = title_x,
)
st.plotly_chart(fig_3, use_container_width = True, theme = streamlit_theme)

# Download original sample dataset

with st.expander('Sample Data'):
  st.write(df.iloc[:500, 1:20:2].style.background_gradient(cmap = cmap))
  csv = df.to_csv(index = False).encode('utf-8')
  st.download_button('Download Data', data = csv, file_name = 'Data.csv', mime = 'text/csv')

## 10 Run Dashboard

In [ ]:
#!streamlit run app.py 

## 11 Save Notebook

In [ ]:
import pathlib
MAIN_PATH = str(pathlib.Path().resolve())

filepath = MAIN_PATH + '\\Data_Analytic_Lookup_v1.ipynb'
filepath

### 11.01 Save as pdf

In [ ]:
#!jupyter nbconvert --to pdf D:\\Python\\Workflow\\01_Data_Analytic\\Data_Analytic_Lookup_v1.ipynb

### 11.02 Save as html
has to remove all widgets first

In [ ]:
#!jupyter nbconvert --to html D:\\Python\\Workflow\\01_Data_Analytic\\Data_Analytic_Lookup_v1.ipynb